In [1]:
import numpy as np
import cv2
import sys
import matplotlib.pyplot as plt
from pathlib import Path

folder = "../fig"

In [5]:
kernel = np.array([[-1, 0, 1],
                   [-2, 0, 2],
                   [-1, 0, 1]], np.float32)

src = cv2.imread(Path(folder, "bamboo.jpg"), cv2.IMREAD_GRAYSCALE)

dst = cv2.filter2D(src, -1, kernel)

cv2.imshow('src', src)
cv2.imshow('dst', dst)

while True:
    if cv2.waitKey() == 27:
        break

cv2.destroyAllWindows()

In [6]:
src = cv2.imread(Path(folder, "bamboo.jpg"), cv2.IMREAD_GRAYSCALE)

kernel = np.array([[-1, 0, 1],
                   [-2, 0, 2],
                   [-1, 0, 1]], np.float32)

dst1 = cv2.filter2D(src, -1, kernel)
dst2 = cv2.filter2D(src, -1, kernel.T)

cv2.imshow('src', src)
cv2.imshow('dst1', dst1)
cv2.imshow('dst2', dst2)

while True:
    if cv2.waitKey() == 27:
        break

cv2.destroyAllWindows()

In [10]:
src = cv2.imread(Path(folder, "son.jpg"), cv2.IMREAD_GRAYSCALE)

dx = cv2.Sobel(src, cv2.CV_32F, 1, 0)
dy = cv2.Sobel(src, cv2.CV_32F, 0, 1)

mag = cv2.magnitude(dx, dy)
mag = np.clip(mag, 0, 255).astype(np.uint8)

ret, dst = cv2.threshold(mag, 150, 255, cv2.THRESH_BINARY)
background = np.zeros((mag.shape[0], mag.shape[1]), np.uint8)
background[mag > 150] = 255

cv2.imshow('src', src)
cv2.imshow('dx', dx)
cv2.imshow('dy', dy)
cv2.imshow('mag', mag)
cv2.imshow('dst', dst)
cv2.imshow('background', background)

cv2.waitKey()
cv2.destroyAllWindows()

In [11]:
src = cv2.imread(Path(folder, "son.jpg"), cv2.IMREAD_GRAYSCALE)
dst = cv2.Canny(src, 150, 180)

cv2.imshow("src", src)
cv2.imshow("dst", dst)

cv2.waitKey()
cv2.destroyAllWindows()

In [18]:
img = cv2.imread(Path(folder, "checkerboard.png"), cv2.IMREAD_GRAYSCALE)

edge = cv2.Canny(img, 100, 200)

lines = cv2.HoughLinesP(edge, 1, np.pi/360, 100, minLineLength=10,
                        maxLineGap=30)

dst = cv2.cvtColor(edge, cv2.COLOR_GRAY2BGR)

back = np.zeros_like(img)
back = cv2.cvtColor(back, cv2.COLOR_GRAY2BGR)

for i in range(lines.shape[0]):
    pt1 = (lines[i][0][0], lines[i][0][1])
    pt2 = (lines[i][0][2], lines[i][0][3])
    cv2.line(back, pt1, pt2, (0, 0, 255), 1, cv2.LINE_AA)

cv2.imshow('img', img)
cv2.imshow('edge', edge)
cv2.imshow("dst", dst)
cv2.imshow("back", back)

cv2.waitKey()
cv2.destroyAllWindows()

In [24]:
src = cv2.imread(Path(folder, "plates.png"))

src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
circles = cv2.HoughCircles(src_gray, cv2.HOUGH_GRADIENT, 1, 50,
                           param1=200, param2=80, minRadius=50, maxRadius=150)

print(circles)

for i in range(circles.shape[1]):
    cx, cy, radius = circles[0][i]
    cv2.circle(src, (int(cx), int(cy)), int(radius), (0, 0, 255), 2, cv2.LINE_AA)

cv2.imshow('src', src)
cv2.imshow("src_gray", src_gray)

cv2.waitKey()
cv2.destroyAllWindows()

[[[113.5  113.5   64.3 ]
  [480.5  147.5  101.  ]
  [204.5  333.5   78.6 ]
  [236.5  180.5   61.5 ]
  [524.5  330.5   59.2 ]
  [668.5  231.5   97.4 ]
  [316.5   76.5   50.25]]]


In [46]:
src = cv2.imread(Path(folder, "symbols.png"))

src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
ret, mask = cv2.threshold(src_gray, 100, 255, cv2.THRESH_BINARY)

cnts, labels, stats, centroids = cv2.connectedComponentsWithStats(mask)
print(cnts)
print(stats[1])

for i in range(1, cnts):
    (x, y, w, h, area) = stats[i]

    if area <= 50:
        continue

    cv2.rectangle(src, (x, y, w, h), (0, 0, 255), 2)

print('counts = ', cnts)

cv2.imshow("src", src)
cv2.imshow("mask", mask)

cv2.waitKey()
cv2.destroyAllWindows()

2
[     0      0    512    512 217198]
counts =  2


In [49]:
src = cv2.imread(Path(folder, "shape.png"))

src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
ret, mask = cv2.threshold(src_gray, 0, 255, cv2.THRESH_OTSU|cv2.THRESH_BINARY_INV)
contours, hierachy = cv2.findContours(mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)

for i in range(len(contours)):
    cv2.drawContours(src, contours, i, (0,0,255), 1)
    cv2.putText(src, str(i), contours[i][0][0], cv2.FONT_HERSHEY_COMPLEX, 1,
                (0,0,255), 1, cv2.LINE_AA)

cv2.imshow('src', src)
cv2.imshow('src_gray', src_gray)
cv2.imshow("mask", mask)
cv2.waitKey()
cv2.destroyAllWindows()

In [51]:
## 기하학적 모멘트 (Hu 불변 모멘트)
# obj = cv2.imread("./fig/spades.png", cv2.IMREAD_GRAYSCALE)
# src = cv2.imread("./fig/symbols.png", cv2.IMREAD_GRAYSCALE)
obj = cv2.imread(Path(folder, "spades.png"), cv2.IMREAD_GRAYSCALE)
src = cv2.imread(Path(folder, "symbols.png"), cv2.IMREAD_GRAYSCALE)



_, obj_bin = cv2.threshold(obj, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)
obj_contours, _ = cv2.findContours(obj_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
obj_pts = obj_contours[0]
_, src_bin = cv2.threshold(src, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)
src_contours, _ = cv2.findContours(src_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)

for pts in src_contours:
    if cv2.contourArea(pts) < 1000:
        continue
    rc = cv2.boundingRect(pts)
    cv2.rectangle(dst, rc, (255, 0, 0), 1)
    dist = cv2.matchShapes(obj_pts, pts, cv2.CONTOURS_MATCH_I3, 0)
    cv2.putText(dst, str(round(dist, 3)), (rc[0], rc[1] - 3), cv2.FONT_HERSHEY_COMPLEX, 0.8,
                (255, 0, 0), 1, cv2.LINE_AA)
    
    if dist < 0.1:
        cv2.rectangle(dst, rc, (0, 0, 255), 2)
        cv2.putText(dst, str(round(dist, 3)), (rc[0], rc[1] - 3), cv2.FONT_HERSHEY_COMPLEX, 0.8,
                (0, 0, 255), 2, cv2.LINE_AA)
        
cv2.imshow("obj", obj)
cv2.imshow("src", src)
cv2.imshow("dst", dst)
# cv2.imshow("obj_bin", obj_bin)

cv2.waitKey()
cv2.destroyAllWindows()

In [61]:
cap = cv2.VideoCapture(Path(folder, "PETS2000.avi"))

if not cap.isOpened():
    print('video open failed')
    sys.exit()

ret, background = cap.read()
background_gray = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
background_gray_G = cv2.GaussianBlur(background_gray, (0,0), 1.)

while True:
    ret, frame = cap.read()

    if not ret:
        break

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_gray_G = cv2.GaussianBlur(frame_gray, (0,0), 1)

    diff_G = cv2.absdiff(frame_gray_G, background_gray_G)

    ret_g, mask_g = cv2.threshold(diff_G, 50, 255, cv2.THRESH_BINARY)
    cnts, labels, stats, centroids = cv2.connectedComponentsWithStats(mask_g)
    for i in range(1, cnts):
        x, y, w, h, area = stats[i]
        if area <= 200:
            continue
        cv2.rectangle(frame, (x, y, w, h), (0,0,255), 2)

    cv2.imshow('frame', frame)
    cv2.imshow('diff_G', diff_G)
    cv2.imshow('mask_g', mask_g)

    if cv2.waitKey(20) == 27:
        break

cv2.destroyAllWindows()
cap.release()

In [ ]:
cap = cv2.VideoCapture(Path(folder, "PETS2000.avi"))

if not cap.isOpened():
    print('video open failed')
    sys.exit()

ret, back = cap.read()
back = cv2.cvtColor(back, cv2.COLOR_BGR2GRAY)
back = cv2.GaussianBlur(back, (0,0), 1.)
fback = back.astype(np.float32)

while True:
    ret, frame = cap.read()

    if not ret:
        print('frame is None')
        break

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_gray = cv2.GaussianBlur(frame_gray, (0,0), 1)
    cv2.accumulateWeighted(frame_gray, fback, 0.01)
    back = fback.astype(np.uint8)

    diff = cv2.absdiff(frame_gray, back)

    ret, mask = cv2.threshold(diff, 50, 255, cv2.THRESH_BINARY)
    cnts, labels, stats, centroids = cv2.connectedComponentsWithStats(mask)
    for i in range(1, cnts):
        x, y, w, h, area = stats[i]
        if area <= 100:
            continue
        cv2.rectangle(frame, (x, y, w, h), (0,0,255), 2)

    cv2.imshow('frame', frame)
    cv2.imshow('diff', diff)
    cv2.imshow('mask', mask)
    cv2.imshow('back', back)

    if cv2.waitKey(20) == 27:
        break

cv2.destroyAllWindows()
cap.release()

In [69]:
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

if not cap.isOpened():
    print('camera open failed')
    cap.release()
    sys.exit()

print(int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

while True:
    ret, frame = cap.read()

    if not ret:
        break

    edge = cv2.Canny(frame, 50, 150)

    cv2.imshow('frame', frame)
    cv2.imshow('edge', edge)

    if cv2.waitKey(10) == 27:
        break

cap.release()
cv2.destroyAllWindows()

640 480


In [70]:
import sys 
import numpy as np
import cv2

def cartoon_filter(img):
    h, w = img.shape[:2]

    blr = cv2.bilateralFilter(img, -1, 20, 7)
    edge = 255 - cv2.Canny(img, 80, 120)
    edge = cv2.cvtColor(edge, cv2.COLOR_GRAY2BGR)

    dst = cv2.bitwise_and(blr, edge)
    dst = cv2.resize(dst, (w, h), interpolation=cv2.INTER_NEAREST)
    
    return dst

def pencil_sketch(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blr = cv2.GaussianBlur(gray, (0, 0), 3)
    dst = cv2.divide(gray, blr, scale=255)

    return dst

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

if not cap.isOpened():
    print('video open failed!')
    sys.exit()

cam_mode = 0

while True:
    ret, frame = cap.read()

    if not ret:
        break

    if cam_mode == 1:
        frame = cartoon_filter(frame)
    elif cam_mode == 2:
        frame = pencil_sketch(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)

    cv2.imshow('frame', frame)
    key = cv2.waitKey(1)

    if key == 27:
        break
    elif key == ord(' '):
        cam_mode += 1
        if cam_mode == 3:
            cam_mode = 0

cap.release()
cv2.destroyAllWindows()